In [7]:
import json
import mysql.connector as mysql
import os
import re
import urllib.request

from pathlib import Path
from PIL import ImageFile

In [8]:
# GENERAL CONFIGS
ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

# == Files config ==
MAIN_FOLDER = r"D:\legal_data\amazon-pqa\d_amazon_pqa_accessories.json"

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp_amazon"

In [9]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

cursor.execute("DROP TABLE IF EXISTS products_tmp_amazon")

In [10]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp_amazon ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 't'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [11]:
def clear_text(text):
    clear_content = re.sub(r'[\t\r\n]', ' ', text)
    clear_content = re.sub(' +',' ', clear_content).lstrip()
    
    return clear_content

In [12]:
folder = Path(f"{MAIN_FOLDER}")

# for file in  os.listdir(folder):
#     file_path = Path(f"{MAIN_FOLDER}\{file}")
#     print(file_path)
    
for line in open(folder, 'r'):
    data = json.loads(line)

    description = clear_text(data['product_description'])
    name = clear_text(data['item_name'])

    query = "INSERT INTO products_tmp_amazon (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
    values = (name, description, "Amazon (PQA)", "f")

    cursor.execute(query, values)
    conn.commit()
